In [1]:
import sys

sys.path.append("../vector_db")

In [2]:
from qdrant.lib.llm import LLM, test
import tiktoken
import openai

sys.path.append("../code")
from util import extract_pdf_to_txt, extract_doi_to_txt, get_string_from_text_file

In [3]:
model = "gpt-3.5-turbo"
tokenizer = tiktoken.encoding_for_model(model)

input = """

"""
len(tokenizer.encode(input))

1

In [4]:
a = [0, 1, 2, 3, 4, 5]
s = -6
i = 0
# a[i:i+s]

In [5]:
def apply_llm(
    input_file: str,
    prompt_file: str = "../llm_prompt.txt",
    # num_chunks: int = -1,
    max_tokens: int = 2000,
    model_token_limit: int = 4096,
):
    model = "gpt-3.5-turbo"
    # Initialize the tokenizer
    tokenizer = tiktoken.encoding_for_model(model)

    input = get_string_from_text_file(input_file)
    # prompt = get_string_from_text_file(prompt_file) + input + "```"
    prompt = get_string_from_text_file(prompt_file)
    # Encode the text_data into token integers
    token_integers = tokenizer.encode(input)

    # Split the token integers into chunks based on max_tokens
    # chunk_size = max_tokens - len(token_integers)
    # if num_chunks != -1:
    #     chunk_size = len(token_integers) // num_chunks
    # else:
    #     chunk_size = model_token_limit - 50
    chunks = [
        token_integers[i : i + max_tokens]
        for i in range(0, len(token_integers), max_tokens)
    ]
    # print("chunk size:", max_tokens)

    # Decode token chunks back to strings
    chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print(f"{len(chunks)} chunks processing...")

    responses = []
    messages = [
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": "To provide the context for the above prompt, I will send you text in parts. When I am finished, I will tell you 'ALL PARTS SENT'. Do NOT answer until you have received all the parts. You will be penalized otherwise.",
        },
    ]

    for i, chunk in enumerate(chunks):
        messages.append({"role": "user", "content": chunk})

        # Check if total tokens exceed the model's limit and remove oldest chunks if necessary
        while (
            sum(len(tokenizer.encode(msg["content"])) for msg in messages)
            > model_token_limit
        ):
            messages.pop(1)  # Remove the oldest chunk

        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = "\n".join([message["content"] for message in messages])
        # response = client.get_response()
        # responses.append(response)
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo", messages=messages
        )
        # chatgpt_response = response.choices[0].message["content"].strip()
        chatgpt_response = response.choices[0].message.content
        responses.append(chatgpt_response)
        print(f"{i}th chunk processed {responses}")

    # Add the final "ALL PARTS SENT" message
    messages.append({"role": "user", "content": "```\nALL PARTS SENT"})
    # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
    # client.prompt = "\n".join([message["content"] for message in messages])
    # response = client.get_response()
    # responses.append(response)
    response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    # final_response = response.choices[0].message["content"].strip()
    final_response = response.choices[0].message.content
    responses.append(final_response)
    return responses

In [16]:
import openai
import os

messages = [
    {
        "role": "user",
        "content": "hi",
    },
]

model = "gpt-35-turbo"
client = openai.AzureOpenAI(
    api_key=os.environ.get("AZURE_API_KEY"),
    api_version=os.environ.get("API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
)
response = client.chat.completions.create(
    model=model,
    messages=messages,
)
response.choices[0].message.content

'Hello! How can I assist you today?'

In [49]:
def apply_llm(
    max_tokens_per_chunk: int = 4000,
    model_token_limit: int = 4096,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    input = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    system_prompt = get_string_from_text_file("../llm_prompt.txt")
    # text = input + prompt
    text = input
    token_integers = tokenizer.encode(text)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(text))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        if i == num_chunks - 1:
            chunk += (
                f"[START PART {i + 1}/{num_chunks}]\n"
                + text[start:end]
                + f"\n[END PART {i + 1}/{num_chunks}]"
            )
            chunk += "\nALL PARTS SENT. Now you can continue processing the request."
        else:
            chunk += (
                f'Do not answer yet. This is just another part of the text I want to send you. Just receive and acknowledge as "Part {i + 1}/{num_chunks} received" and wait for the next part.\n[START PART {i + 1}/{num_chunks}]\n'
                + decoded_chunk
                + f"\n[END PART {i + 1}/{num_chunks}]"
            )
            chunk += f'\nRemember not answering yet. Just acknowledge you received this part with the message "Part {i + 1}/{num_chunks} received" and wait for the next part.'

        chunks.append(chunk)

    responses = []

    model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    # model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    for i, chunk in enumerate(chunks):
        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = chunk
        # response = client.get_response(max_tokens=1000)
        # responses.append(response)
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": chunk,
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if verbose:
            print(response)

    # print(responses)


apply_llm()

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
3 chunks processing...


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4445 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
# apply_llm(
#     input_file="../text/2020 ESC Guidelines for the diagnosis and management of atrial fibrillation developed in collaboration with the European Association for Cardio-Thoracic Surgery (EACTS).txt",
# )
apply_llm()

In [37]:
input = get_string_from_text_file(
    "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
)
prompt = get_string_from_text_file("../llm_prompt.txt")
# input = "hi"
# input = input * 100
with open("../llm_prompt_temp.txt", "w") as file:
    file.write(prompt + input + "\n```")

tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
print(len(tokenizer.encode(get_string_from_text_file("../llm_prompt_temp.txt"))))
client = LLM("../llm_prompt_temp.txt", base="azure", use_model="gpt4")
client.get_response(max_tokens=2000)

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

reading data from ../llm_prompt_temp.txt...
string successfully retrieved.

905051


KeyboardInterrupt: 